In [ ]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Github/cis5810

/content/drive/MyDrive/Github


In [36]:
"""
Function to determine camera shot movement (pan, tilt, zoom, or static)
"""

def detect_camera_movement(video_path):
    cap = cv2.VideoCapture(video_path)

    ret, prev_frame = cap.read()
    if not ret:
        print("Error reading video file")
        return

    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

    movement = {'pan': 0, 'tilt': 0, 'zoom': 0}

    while True:
        ret, next_frame = cap.read()
        if not ret:
            break

        next_gray = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)

        flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None,
                                            0.5, 3, 15, 3, 5, 1.2, 0)
        # Horizontal flow
        flow_x = flow[..., 0]
        # Vertical flow
        flow_y = flow[..., 1]

        mean_flow_x = np.mean(flow_x)
        mean_flow_y = np.mean(flow_y)

        magnitude, angle = cv2.cartToPolar(flow_x, flow_y, angleInDegrees=True)
        mean_magnitude = np.mean(magnitude)

        # Case 1: Pan
        if abs(mean_flow_x) > abs(mean_flow_y):
            movement_type = "Pan" if mean_flow_x > 0 else "Reverse Pan"
            movement['pan'] += 1
        # Case 2: Tilt (vertical movement)
        elif abs(mean_flow_y) > abs(mean_flow_x):
            movement_type = "Tilt" if mean_flow_y > 0 else "Reverse Tilt"
            movement['tilt'] += 1
        # Case 3: Zoom
        else:
            if np.median(flow_x) < 0 and np.median(flow_y) < 0:
                movement_type = "Zoom In"
                movement['zoom'] += 1
            elif np.median(flow_x) > 0 and np.median(flow_y) > 0:
                movement_type = "Zoom Out"
                movement['zoom'] += 1
            else:
                movement_type = "Static/No significant movement"

        print(f"Detected Movement: {movement_type}")
        # Show the optical flow map (optional)
        hsv = np.zeros_like(prev_frame)
        hsv[..., 1] = 255
        hsv[..., 0] = angle / 2  # Hue corresponds to the direction of flow
        hsv[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
        flow_map = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        cv2_imshow(next_frame)
        cv2_imshow(flow_map)

        # Update previous frame
        prev_gray = next_gray

        # Break loop on 'q' key press
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    print("Overall Movement Detected: ", movement)

In [37]:
! git add .

In [ ]:
! git branch